In [1]:
import pandas as pd
import pickle
import sys
from add_features import add_features
import import_ipynb
from feeds import BackFeed,DataFeed
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from synfeed.ipynb
importing Jupyter notebook from india_calendar.ipynb
importing Jupyter notebook from featfuncs.ipynb


In [2]:
feed = pickle.load(open('../algodata/realdata/datafeed_alldata_True_False.pkl', 'rb'))

In [3]:
with open('additional_utils/cols.pkl', 'rb') as f:
        d = pickle.load(f)
        
# d['discrete_cols']+d['pre_discrete_cols']
imp_cols = d['imp_cols']
cols_to_use = d['cols_to_use']
col_counts = d['col_counts']
col_counts = {k:v for k,v in sorted(col_counts.items(), key=lambda x:-x[1])}
col_counts

{'Open-Close_ma_1-Open-Close_ma_10_val': 5,
 'Volume_ma_10_val': 4,
 'Close_slope_1_polarity': 3,
 'Open-Close_ma_1-Open-Close_ma_50_val': 3,
 'BBP_5_2.0_slope_1_changelen_val': 3,
 'Open-Close_slope_1_val': 2,
 'BBM_5_2.0_ma_1-BBM_5_2.0_ma_20_changelen_val': 2,
 'BBL_5_2.0_ma_10_slope_15_val': 2,
 'MACDs_12_26_9_ma_20_changelen_val': 1,
 'Open-High_ma_5-Open-High_ma_20_changelen_val': 1,
 'BBL_5_2.0_ma_5_slope_1_changelen_polarity': 1,
 'High_n-Low_n_ma_50_changelen_val': 1,
 'High_n-Low_n_ma_20_slope_15_changelen_val': 1,
 'Open_n-Close_n_ma_10_val': 1,
 'Volume_ma_5_slope_5_val': 1,
 'Close_n_slope_1_polarity': 1,
 'Low_ma_5_slope_3_polarity': 1,
 'Open_n_ma_10_slope_5_changelen_polarity': 1,
 'RSI_14_ma_1_val': 1,
 'Open-Close_polarity': 1,
 'Open-High_ma_1-Open-High_ma_50_val': 1,
 'Open-Close_ma_1-Open-Close_ma_20_val': 1,
 'Close_n_ma_1-Close_n_ma_50_changelen_polarity': 1,
 'High_n_ma_1-High_n_ma_20_changelen_polarity': 1,
 'Open-High_ma_1-Open-High_ma_50_polarity': 1}

In [4]:
features = list(col_counts.keys())[:5]
features

['Open-Close_ma_1-Open-Close_ma_10_val',
 'Volume_ma_10_val',
 'Close_slope_1_polarity',
 'Open-Close_ma_1-Open-Close_ma_50_val',
 'BBP_5_2.0_slope_1_changelen_val']

In [5]:
feature_mapper = {
    'val': {0: 'low', 1: 'medium', 2: 'high'},
    'polarity': {-1: 'negative', 1: 'positive'}
}

In [6]:
df = feed.data['CYIENT.NS']
df

,Open,High,Low,Close,Volume,Datetime,ticker,Date,datetime,row_num,...,BBB_5_2.0,BBP_5_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,VWAP_D,MOM_30,CMO_14,error,sym
datetime,,,,,,,,,,,,,,,,,,,,,
2022-02-02 09:15:00+05:30,986.549988,986.549988,977.049988,978.849976,3452.0,2022-02-02 09:15:00+05:30,CYIENT.NS,02-Feb-2022,2022-02-02 09:15:00+05:30,0,...,0.267309,0.714135,0.005119,0.000673,0.004446,1.037901,0.027566,41.341582,-1,2
2022-02-02 09:20:00+05:30,978.700012,982.900024,976.500000,982.900024,5729.0,2022-02-02 09:20:00+05:30,CYIENT.NS,02-Feb-2022,2022-02-02 09:20:00+05:30,1,...,0.793277,0.971161,0.005446,0.000800,0.004646,1.037868,0.032116,50.223128,-1,2
2022-02-02 09:25:00+05:30,981.950012,985.500000,981.299988,984.049988,3832.0,2022-02-02 09:25:00+05:30,CYIENT.NS,02-Feb-2022,2022-02-02 09:25:00+05:30,2,...,1.050253,0.849608,0.005738,0.000873,0.004864,1.038750,0.034444,52.425743,-1,2
2022-02-02 09:30:00+05:30,984.950012,985.700012,981.049988,983.900024,2270.0,2022-02-02 09:30:00+05:30,CYIENT.NS,02-Feb-2022,2022-02-02 09:30:00+05:30,3,...,0.936252,0.731729,0.005888,0.000819,0.005069,1.039054,0.033333,51.484364,-1,2
2022-02-02 09:35:00+05:30,983.900024,988.900024,980.849976,986.000000,5968.0,2022-02-02 09:35:00+05:30,CYIENT.NS,02-Feb-2022,2022-02-02 09:35:00+05:30,4,...,0.963617,0.801888,0.006116,0.000837,0.005278,1.040048,0.032804,55.617954,-1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 15:06:00+05:30,1024.750000,1025.000000,1023.099976,1024.650024,4613.0,2021-12-31 15:06:00+05:30,CYIENT.NS,31-Dec-2021,2021-12-31 15:06:00+05:30,145,...,0.526468,0.300009,-0.000326,0.000165,-0.000491,1.021888,-0.008710,-7.999679,0,2
2021-12-31 15:11:00+05:30,1024.650024,1025.000000,1021.950012,1022.299988,8802.0,2021-12-31 15:11:00+05:30,CYIENT.NS,31-Dec-2021,2021-12-31 15:11:00+05:30,146,...,0.651184,0.132846,-0.000570,-0.000063,-0.000507,1.021810,-0.010353,-19.134176,0,2
2021-12-31 15:16:00+05:30,1022.549988,1023.049988,1019.200012,1019.549988,17995.0,2021-12-31 15:16:00+05:30,CYIENT.NS,31-Dec-2021,2021-12-31 15:16:00+05:30,147,...,0.788036,0.053503,-0.000973,-0.000373,-0.000600,1.021552,-0.009158,-29.835584,0,2


In [7]:
df, _, _ = add_features(df, columns_to_use=cols_to_use)
df

,Open,High,Low,Close,Volume,Datetime,ticker,Date,datetime,row_num,...,High-Low_ma_20-High-Low_ma_50_changelen_val,High-Low_ma_20-High-Low_ma_50_changelen_polarity,Open-Close_ma_20-Open-Close_ma_50_changelen_val,Open-Close_ma_20-Open-Close_ma_50_changelen_polarity,Open_n-Close_n_ma_20-Open_n-Close_n_ma_50_changelen_val,Open_n-Close_n_ma_20-Open_n-Close_n_ma_50_changelen_polarity,High_n-Low_n_ma_20-High_n-Low_n_ma_50_changelen_val,High_n-Low_n_ma_20-High_n-Low_n_ma_50_changelen_polarity,Open-High_ma_20-Open-High_ma_50_changelen_val,Open-High_ma_20-Open-High_ma_50_changelen_polarity
datetime,,,,,,,,,,,,,,,,,,,,,
2022-02-02 09:15:00+05:30,986.549988,986.549988,977.049988,978.849976,3452.0,2022-02-02 09:15:00+05:30,CYIENT.NS,02-Feb-2022,2022-02-02 09:15:00+05:30,0,...,1,-1,1,-1,1,-1,1,-1,1,-1
2022-02-02 09:20:00+05:30,978.700012,982.900024,976.500000,982.900024,5729.0,2022-02-02 09:20:00+05:30,CYIENT.NS,02-Feb-2022,2022-02-02 09:20:00+05:30,1,...,1,1,2,1,2,1,1,1,2,1
2022-02-02 09:25:00+05:30,981.950012,985.500000,981.299988,984.049988,3832.0,2022-02-02 09:25:00+05:30,CYIENT.NS,02-Feb-2022,2022-02-02 09:25:00+05:30,2,...,1,1,2,1,2,1,1,1,2,1
2022-02-02 09:30:00+05:30,984.950012,985.700012,981.049988,983.900024,2270.0,2022-02-02 09:30:00+05:30,CYIENT.NS,02-Feb-2022,2022-02-02 09:30:00+05:30,3,...,1,1,2,1,2,1,1,1,2,1
2022-02-02 09:35:00+05:30,983.900024,988.900024,980.849976,986.000000,5968.0,2022-02-02 09:35:00+05:30,CYIENT.NS,02-Feb-2022,2022-02-02 09:35:00+05:30,4,...,2,1,2,1,2,1,2,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 15:06:00+05:30,1024.750000,1025.000000,1023.099976,1024.650024,4613.0,2021-12-31 15:06:00+05:30,CYIENT.NS,31-Dec-2021,2021-12-31 15:06:00+05:30,145,...,1,1,0,-1,0,-1,1,1,2,1
2021-12-31 15:11:00+05:30,1024.650024,1025.000000,1021.950012,1022.299988,8802.0,2021-12-31 15:11:00+05:30,CYIENT.NS,31-Dec-2021,2021-12-31 15:11:00+05:30,146,...,2,1,0,-1,0,-1,2,1,1,-1
2021-12-31 15:16:00+05:30,1022.549988,1023.049988,1019.200012,1019.549988,17995.0,2021-12-31 15:16:00+05:30,CYIENT.NS,31-Dec-2021,2021-12-31 15:16:00+05:30,147,...,2,1,2,1,2,1,2,1,0,-1


In [8]:
def convert_to_candles(df):
    df['candle'] = ''
    open, close = df['Open'], df['Close']
    green = (close - open)>0
    df.loc[green, 'candle'] = 'G'
    df.loc[~green, 'candle'] = 'R'

    diff = np.abs((close-open).values)
    low_thresh, high_thresh = np.quantile(diff, 0.25), np.quantile(diff, 0.75)
    df.loc[(diff <= low_thresh), 'candle'] += 'L'
    df.loc[(diff > low_thresh) & (diff < high_thresh), 'candle'] += 'M'
    df.loc[(diff >= high_thresh), 'candle'] += 'H'
    return df

In [9]:
candle_abbreviations = {'R': 'Red', 'G': 'Green', 'H': 'High', 'M': 'Medium', 'L': 'Low'}

In [13]:
def expand_candles(candle_values):
    expanded_values = []
    for value in candle_values:
        curr_expansion = ''
        for char in value:
            curr_expansion += candle_abbreviations[char] + " "
        expanded_values.append(curr_expansion.strip())
    return expanded_values

In [10]:
def reorder_prompt(prompt) -> str:
    if type(prompt) == str:
        prompt_list = prompt.split("\n")
    else:
        prompt_list = prompt
        
    new_prompt = np.random.shuffle(prompt_list)
    return "\n".join(new_prompt)
    

In [29]:
def find_similar(sample, , k=5):
    pass

In [ ]:
def find_random(sample, df, k=5):
    

In [34]:
def get_current(df, idx, last_k=5, closecol='candle', features=features, expand=False):
    num = df.iloc[idx, :]['row_num']
    if num < last_k: return None
    
    prev_5 = df.iloc[idx-last_k:idx, :][closecol].values
    if expand: prev_5 = expand_candles(prev_5)
    prev_5_str = ' '.join(prev_5)
    
    feature_vals = df.iloc[idx][features].values.tolist()
    # print(feature_vals, features)
    f_list = []
    for idx, feature in enumerate(features):
        for key in feature_mapper:
            if feature.endswith(key):
                f_list.append(f'f{idx}: {feature_mapper[key][feature_vals[idx]]}')
                
    feature_str = ' '.join(f_list)
        
    complete_str = prev_5_str + ' ' + feature_str
    return complete_str

In [35]:
def get_future(df, idx, next_k=5, closecol='candle', expand=False):
    num = df.iloc[idx, :]['row_num']
    if num + next_k > 75: return None
    
    next_5 = df.iloc[idx:idx+next_k, :][closecol].values
    if expand: next_5 = expand_candles(next_5)
    next_5_str = ' '.join(next_5)
    
    return next_5_str

In [36]:
df['row_num'] = df['row_num'].apply(lambda x: x-75 if x>74 else x)
df = convert_to_candles(df)

In [31]:
# all_samples = {}

# for idx in range(len(df)):
#     all_samples[idx] = get_current(df, idx)

In [32]:
def generate_prompt(df, idx):
    current_sample = get_current(df, idx, expand=True)
    similar_samples_idxs = find_random(current_sample)
    prompt_samples_curr = [all_samples[i] for i in similar_samples_idxs]
    prompt_samples_future = [get_future(df, i) for i in similar_samples_idxs]
    prompt_samples = [prompt_samples_curr[i]+ " "+ prompt_samples_future[i] for i in range(len(prompt_samples_curr))]
    prompt_samples_str = "\n".join(prompt_samples)
    prompt = prompt_samples_str + "\n" + current_sample

In [37]:
get_current(df, 60, expand=True)

'Red Medium Green High Red Medium Red Medium Green Low f0: low f1: medium f2: positive f3: low f4: high'

In [38]:
get_future(df, 60, expand=True)


'Green Medium Green Medium Red Low Green Medium Red Medium'

In [ ]:
## No of tokens: No of prompt samples and the number of past times steps and the number of feature time steps 
## Legend: One description for each feature
## Recursive tree
## Testing: Real time add features in the agent
## Test on the OpenAI playground
## Try the RLAgent with the pre-discretized features

Open-Close_ma_1-Open-Close_ma_10_val: The difference between the moving averages of window sizes 1 and 10 of Opening Price - Closing Price for a stock

Volume_ma_10_val: The moving average of volume with window size 10

Close_slope_1_polarity: The difference between the current closing price and the previous closing price

Open-Close_ma_1-Open-Close_ma_50_val: The difference between the moving averages of window sizes 1 and 10 of Opening Price - Closing Price

BBP_5_2.0_slope_1_changelen_val: The length for which the slope of BBP_5_2.0 has been increasing (include polarity)

In [ ]:
# Most similar
# Most recent (window of 5)
# Random

# For testing, make them into ordinal and MSE
# Do it for 5-10 samples

# Model: gpt-3.5-turbo, gpt-3 (text-davinci-003)

# Give a small description of the task, End with 'Predict the next 5'

# Run multiple times on synthetic data and average the results

# Summarize the results of the first part
# Distill the new models

In [ ]:
# Legend: 100 tokens
# Sample with Future: 40 tokens